In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
import plotly.graph_objs as go

In [3]:
print(__version__)

3.5.0


In [4]:
import pandas as pd

In [5]:
zetus = pd.read_csv("https://raw.githubusercontent.com/gautsi/gen-purpose-repo/master/zetus.csv")

In [6]:
zetus_dedup = zetus.drop_duplicates()

In [7]:
len(zetus)

208

In [8]:
len(zetus_dedup.drop_duplicates())

204

In [9]:
zetus_dedup["time_"] = pd.to_datetime(zetus_dedup.time)

/home/gautam_sisodia/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



separate same time feedings

In [10]:
zetus_dedup["prev_dur"] = zetus_dedup.sort_values("time_").groupby(["time_"])["dur"].shift(1)

/home/gautam_sisodia/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [11]:
zetus_dedup["fixed_time_"] = zetus_dedup.apply(
    lambda x: x.time_ + pd.Timedelta(minutes = (x.prev_dur if pd.notna(x.prev_dur) else 0)),
    axis = 1)

/home/gautam_sisodia/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [12]:
init_notebook_mode(connected=True)

In [13]:
zetus_dedup.dtypes

time                   object
dur                     int64
type                   object
time_          datetime64[ns]
prev_dur              float64
fixed_time_    datetime64[ns]
dtype: object

In [14]:
import colorlover as cl

In [37]:
colorscale = cl.scales['5']['qual']['Dark2']
light_colorscale = cl.scales['5']['qual']['Set2']

In [16]:
zetus_dedup["day"] = zetus_dedup.fixed_time_.dt.date

/home/gautam_sisodia/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [17]:
zetus_dedup["time_of_day"] = pd.to_datetime(zetus_dedup.fixed_time_.dt.time.astype("str").map(lambda x: "01/01/19 {}".format(x)))

/home/gautam_sisodia/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [18]:
feed_data = [
    go.Bar(
        x = zetus_dedup[zetus_dedup["type"] == side].sort_values("fixed_time_").fixed_time_,
        y = 2 * (i - 0.5) * zetus_dedup[zetus_dedup["type"] == side].sort_values("fixed_time_").dur,
        name = side,
        width = 4 * 1000 * 60,
        marker = {"color": colorscale[i]}) for i, side in enumerate(["L", "R"])
]

output_data = [
    go.Bar(
        x = zetus_dedup[zetus_dedup["type"] == output].sort_values("fixed_time_").fixed_time_,
        y = len(zetus_dedup[zetus_dedup["type"] == output]) * [60],
        name = output,
        base = -30,
        width = 4 * 1000 * 60,
        marker = {"color": colorscale[i + 2]}) for i, output in enumerate(["S", "W"])
]

layout = go.Layout()

fig = go.Figure(data = feed_data + output_data, layout = layout)

iplot(fig)

In [19]:
plot(fig)

'file:///home/gautam_sisodia/gen-purpose-repo/zetus/temp-plot.html'

In [20]:
from plotly import tools

In [21]:
days = zetus_dedup.day.unique()

In [22]:
days

array([datetime.date(2019, 1, 18), datetime.date(2019, 1, 19),
       datetime.date(2019, 1, 20), datetime.date(2019, 1, 21),
       datetime.date(2019, 1, 22), datetime.date(2019, 1, 23),
       datetime.date(2019, 1, 24), datetime.date(2019, 1, 25),
       datetime.date(2019, 1, 26)], dtype=object)

In [68]:
feed_data = {
    day: [
        go.Bar(
            x = zetus_dedup[(zetus_dedup["day"] == day) & (zetus_dedup["type"] == side)].sort_values("fixed_time_").time_of_day,
            y = len(zetus_dedup[(zetus_dedup["day"] == day) & (zetus_dedup["type"] == side)]) * [1],
            width = 1000 * 60 * zetus_dedup[(zetus_dedup["day"] == day) & (zetus_dedup["type"] == side)].sort_values("fixed_time_").dur,
            name = side,
            legendgroup = side,
            showlegend = day == days[0],
            text = zetus_dedup[
                (zetus_dedup["day"] == day) & (zetus_dedup["type"] == side)].sort_values("fixed_time_").apply(
                lambda x: "{}, {}".format(x.time_, x.dur), axis = 1),
            # width = 4 * 1000 * 60,
            offset = 0,
            marker = {"color": colorscale[i]}) for i, side in enumerate(["L", "R"])] + [
        go.Scatter(
            x = zetus_dedup[(zetus_dedup["day"] == day) & (zetus_dedup["type"] == output)].sort_values("fixed_time_").time_of_day,
            y = len(zetus_dedup[(zetus_dedup["day"] == day) & (zetus_dedup["type"] == output)]) * [i],
            name = output,
            legendgroup = output,
            showlegend = day == days[0],
            mode = "markers",
            marker = {"color": colorscale[i + 2], "size": 10}) for i, output in enumerate(["S", "W"])] for day in days}

In [69]:
fig = tools.make_subplots(
    rows = len(days),
    cols = 1,
    shared_xaxes = True,
    shared_yaxes = True,
    vertical_spacing = 0.02,
    subplot_titles = ["First Days Input/Output"]
)

for i, day in enumerate(days):
    for bar in feed_data[day]:
        fig.append_trace(bar, i + 1, 1)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]
[ (9,1) x1,y9 ]



In [70]:
import datetime as dt

In [71]:
fig["layout"].update(
    # height = 400,
    # width = 1000,
    xaxis = {
        "tickformat": "%H:%M",
        "range": [
            dt.datetime.strptime("01/01/19 00:00", "%m/%d/%y %H:%M"),
            dt.datetime.strptime("01/01/19 23:59", "%m/%d/%y %H:%M")]
    },
    hovermode = "closest",
    plot_bgcolor = "#DCDCDC")

for i in range(len(days)):
    # fig["layout"]["yaxis{}".format(i + 1)]["showgrid"] = False
    fig["layout"]["yaxis{}".format(i + 1)]["tick0"] = 0
    fig["layout"]["yaxis{}".format(i + 1)]["dtick"] = 1
    fig["layout"]["yaxis{}".format(i + 1)]["range"] = [0, 1]    
    fig["layout"]["yaxis{}".format(i + 1)]["showticklabels"] = False
    fig["layout"]["yaxis{}".format(i + 1)]["zeroline"] = False
    fig["layout"]["yaxis{}".format(i + 1)]["title"] = i + 1

In [72]:
iplot(fig)

In [133]:
tools.make_subplots?

In [62]:
zetus_dedup["mod_dur"] = zetus_dedup.apply(lambda x: x.dur if x["type"] in ["R", "L"] else 1, axis = 1)

/home/gautam_sisodia/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [46]:
zetus_dedup.head()

,time,dur,type,time_,day,mod_dur
0,01/18/19 10:00,15,R,2019-01-18 10:00:00,2019-01-18,15
1,01/18/19 10:00,10,L,2019-01-18 10:00:00,2019-01-18,10
2,01/18/19 12:15,15,R,2019-01-18 12:15:00,2019-01-18,15
3,01/18/19 13:00,0,S,2019-01-18 13:00:00,2019-01-18,1
4,01/18/19 14:35,10,L,2019-01-18 14:35:00,2019-01-18,10


In [47]:
zetus_stats = zetus_dedup.groupby(["day", "type"], as_index = False)["mod_dur"].sum().sort_values("day").pivot(index = "day", columns = "type", values = "mod_dur")

In [48]:
zetus_num_feeds = zetus_dedup[zetus_dedup["type"].isin(["L", "R"])].groupby(["day", "type"], as_index = False)["mod_dur"].count().sort_values("day").pivot(index = "day", columns = "type", values = "mod_dur")

In [49]:
zetus_num_feeds

type,L,R
day,,
2019-01-18,4,4
2019-01-19,9,11
2019-01-20,8,9
2019-01-21,8,8
2019-01-22,7,6
2019-01-23,7,5
2019-01-24,6,7
2019-01-25,4,6
2019-01-26,6,5


In [50]:
zetus_stats

type,L,R,S,W
day,,,,
2019-01-18,55,75,1,1
2019-01-19,147,160,1,5
2019-01-20,145,120,4,1
2019-01-21,100,120,4,8
2019-01-22,80,135,6,6
2019-01-23,100,90,7,7
2019-01-24,115,100,5,7
2019-01-25,85,125,3,7
2019-01-26,105,75,6,5


In [51]:
zetus_stats.reset_index()

type,day,L,R,S,W
0,2019-01-18,55,75,1,1
1,2019-01-19,147,160,1,5
2,2019-01-20,145,120,4,1
3,2019-01-21,100,120,4,8
4,2019-01-22,80,135,6,6
5,2019-01-23,100,90,7,7
6,2019-01-24,115,100,5,7
7,2019-01-25,85,125,3,7
8,2019-01-26,105,75,6,5


In [52]:
zetus_stats_2 = zetus_stats.reset_index().merge(
    right = zetus_num_feeds.rename(columns = {"L": "num_L", "R": "num_R"}).reset_index(),
    on = ["day"],
    how = "left")

In [53]:
zetus_stats_2["avg_L"] = zetus_stats_2.L / zetus_stats_2.num_L
zetus_stats_2["avg_R"] = zetus_stats_2.R / zetus_stats_2.num_R

In [54]:
zetus_stats_2

type,day,L,R,S,W,num_L,num_R,avg_L,avg_R
0,2019-01-18,55,75,1,1,4,4,13.750000,18.750000
1,2019-01-19,147,160,1,5,9,11,16.333333,14.545455
2,2019-01-20,145,120,4,1,8,9,18.125000,13.333333
3,2019-01-21,100,120,4,8,8,8,12.500000,15.000000
4,2019-01-22,80,135,6,6,7,6,11.428571,22.500000
5,2019-01-23,100,90,7,7,7,5,14.285714,18.000000
6,2019-01-24,115,100,5,7,6,7,19.166667,14.285714
7,2019-01-25,85,125,3,7,4,6,21.250000,20.833333
8,2019-01-26,105,75,6,5,6,5,17.500000,15.000000
